# Symmetric Key

## Introduction

Symmetric key encryption involves the same key being used to encrypt and decrypt:

![Alt text](graphics/g_sym_04.png "Symmetric Key")

 Apart from ECB mode, we also normally apply a salt value known as an IV (Initialization Vector). The main modes are ECB (Electronic Code Book), CBC (Cipher Block Chaining), CTR (Counter), OFB (Output Feedback) and GCM (Galois/Counter Mode). Overall, Bob and Alice will share the same encryption key, and covert plaintext into ciphertext for the encryption key, and then from ciphertext to plaintext when we decrypt. Overall, Bob and Alice share the same key, and where Eve will find it difficult to find the key.
 
 ## Block cipher

 With a block cipher, we take the plaintext data, and split it into blocks. This block size can vary between different ciphers, but will typically be 128 bits (16 bytes). Some older symmetric key methods, such as DES and 3DES, use a 64-bit block size (8 bytes). 
 
 ![Alt text](graphics/g_sym_08.png "Symmetric Key")
 
 In the following, we define a message, and then the number of bytes per block (n).  We can then check the value of n to change the size of the block. For the last block, we will pad with the space character:

In [11]:
# https://asecuritysite.com/encryption/blk
import sys

BLOCKSIZE = 8
n=BLOCKSIZE
message="Hello how are you?"

print ("Message:",message)
print ("Bytes per block:",n)
print ("\nBlocks ...")

message = [message[i: i + n] for i in range(0, len(message), n)]

lengthOfLastBlock = len(message[len(message)-1])

if ( lengthOfLastBlock < BLOCKSIZE):
	for i in range(lengthOfLastBlock, BLOCKSIZE):
 		message[len(message)-1] += " "

i=0
for b in message:
	print ('Block [',i,']',b)
	i=i+1


Message: Hello how are you?
Bytes per block: 8

Blocks ...
Block [ 0 ] Hello ho
Block [ 1 ] w are yo
Block [ 2 ] u?      


> For a message of "This is a test message.". How many blocks will be used for a 64-bit block size?
> For a message of "This is a test message.". How many blocks will be used for a 128-bit block size?

  
 ### Padding with PKCS7
In the last block, we may not have enough bytes to fill it up. We thus need to add padding bytes before the encryption, and then remove after we decrypt. The most typical way that we pad is with PKCS7, and where we find out the number of bytes that we need to pad for the last block, and then take that value and repeat it for the number of bytes we need to pad. in the following, we have size as the number of bits in a block, and a message. For "abc" and for a block size of 128 bits (16 bytes), we will have 13 filling bytes. The value of 13 is represented as 0xd:

In [12]:
# https://asecuritysite.com/hazmat/hashnew28

from cryptography.hazmat.primitives import padding
import sys
import binascii

size=128
message="abcde"


print ("Message: ",message)
print ("Block size: ",size)

message=message.encode()
padder = padding.PKCS7(size).padder()
padded_data = padder.update(message)

padded_data += padder.finalize()
print ("Padded: ",binascii.hexlify(padded_data).decode())

unpadder = padding.PKCS7(size).unpadder()
data = unpadder.update(padded_data)

unpadded = data + unpadder.finalize()

print ("Unpadded: ",unpadded.decode())

Message:  abcde
Block size:  128
Padded:  61626364650b0b0b0b0b0b0b0b0b0b0b
Unpadded:  abcde


> For a message of "hello" and a block size of 128 bits. What is the padding value?"
> For a message of "The boy stood on the burning deck!" and a block size of 128 bits. What is the padding value?"
> What will happen when we have 18 characters in the plaintext. Will we have one block or two, and what will the padding value be?

 ## AES encryption
 
 The most popular symmetric key method is AES (Advanced Encryption Standard), and which was defined as a standard by NIST. It has a 128-bit block (16 bytes) and can use a 128-bit, 192-bit or 256-bit encryption key.  We then have a number of modes that can be used. The most basic mode is ECB (Electronic Code Book), and which basically just applies the encryption key to each block:

  ![Alt text](graphics/g_sym_13.png "Symmetric Key")

In the following, we apply create AES encryption with ECB mode:


In [13]:
# 02_01.py
# https://asecuritysite.com/hazmat/hashnew4
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes 
from cryptography.hazmat.primitives import padding
import sys
from cryptography.hazmat.backends import default_backend

import binascii

def go_encrypt(msg,method,mode):
  cipher = Cipher(method, mode,backend=default_backend())
  encryptor = cipher.encryptor()
  ct = encryptor.update(msg) + encryptor.finalize()
  return (ct)


def go_decrypt(ct,method,mode):
  cipher = Cipher(method, mode,backend=default_backend())
  decryptor = cipher.decryptor()
  return (decryptor.update(ct) + decryptor.finalize())


def pad(data,size=128):
  padder = padding.PKCS7(size).padder()
  padded_data = padder.update(data)
  padded_data += padder.finalize()
  return(padded_data)

def unpad(data,size=128):
  padder = padding.PKCS7(size).unpadder()
  unpadded_data = padder.update(data)
  unpadded_data += padder.finalize()
  return(unpadded_data)


key = os.urandom(32)

msg=b"Hello"

print ("Message:\t",msg.decode())
print ("Key:\t",binascii.b2a_hex(key))

padded_data=pad(msg)


print ("\n\n=== AES ECB === ")
cipher=go_encrypt(padded_data,algorithms.AES(key), modes.ECB())

plain=go_decrypt(cipher,algorithms.AES(key), modes.ECB())
data=unpad(plain)

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")

Message:	 Hello
Key:	 b'dd87fcbf7b6d0a43b339e4cab145e8c3520649a6fcbf7cebdc1af053999cd20c'


=== AES ECB === 
Cipher:  b'f52c9d5089181d25bf57221d49a2d256'
Decrypted: Hello


> With a message of "Hello123". What is the ciphertext in hexadecimal?
> If you encrypt the same message, do the message change?

## Additional Modes
The problem with ECB, is that the output cipher is always the same for the same plaintext input. To overcome this, we can add an Initialisation Vector (IV) or salt. This is normally a random value that is greater than 64 bits. The cipher is then stored with the IV, and when we decrypt, we need both the encryption key and the salt value. With CBC (Cipher Block Chaining), we add salt into the first block encryption, and then chain the output of this into the next block, and so on:

![Alt text](graphics/g_sym_12.png "Symmetric Key")

In the following, we use the cipher modes of CBC, OBF, CFB and CTR, and where we add a random salt value:

In [14]:
# 02_02.py
# https://asecuritysite.com/hazmat/hashnew4
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes 
from cryptography.hazmat.primitives import padding
import sys
from cryptography.hazmat.backends import default_backend

import binascii

def go_encrypt(msg,method,mode):
  cipher = Cipher(method, mode,backend=default_backend())
  encryptor = cipher.encryptor()
  ct = encryptor.update(msg) + encryptor.finalize()
  return (ct)


def go_decrypt(ct,method,mode):
  cipher = Cipher(method, mode,backend=default_backend())
  decryptor = cipher.decryptor()
  return (decryptor.update(ct) + decryptor.finalize())


def pad(data,size=128):
  padder = padding.PKCS7(size).padder()
  padded_data = padder.update(data)
  padded_data += padder.finalize()
  return(padded_data)

def unpad(data,size=128):
  padder = padding.PKCS7(size).unpadder()
  unpadded_data = padder.update(data)
  unpadded_data += padder.finalize()
  return(unpadded_data)


key = os.urandom(32)
iv = os.urandom(16)
msg=b"Hello"

print ("Message:\t",msg.decode())
print ("Key:\t",binascii.b2a_hex(key))
print ("IV:\t",binascii.b2a_hex(iv))

padded_data=pad(msg)



print ("=== AES CBC === ")
cipher=go_encrypt(padded_data,algorithms.AES(key), modes.CBC(iv))

plain=go_decrypt(cipher,algorithms.AES(key), modes.CBC(iv))
data=unpad(plain)

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")


print ("=== AES OFB === ")
cipher=go_encrypt(padded_data,algorithms.AES(key), modes.OFB(iv))

plain=go_decrypt(cipher,algorithms.AES(key), modes.OFB(iv))
data=unpad(plain)

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")

print ("=== AES CFB === ")
cipher=go_encrypt(padded_data,algorithms.AES(key), modes.CFB(iv))

plain=go_decrypt(cipher,algorithms.AES(key), modes.CFB(iv))
data=unpad(plain)

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")

print ("=== AES CTR === ")
cipher=go_encrypt(padded_data,algorithms.AES(key), modes.CTR(iv))

plain=go_decrypt(cipher,algorithms.AES(key), modes.CTR(iv))
data=unpad(plain)

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")


Message:	 Hello
Key:	 b'4cf24c4b7435aeb48e92f93f614dccc4d4e2b712867b1cabd0fafb80f1ab7d6b'
IV:	 b'e476433e82a1bf574af60eff91832e52'
=== AES CBC === 
Cipher:  b'bdf3405006a681a750ed2f692e58278c'
Decrypted: Hello
=== AES OFB === 
Cipher:  b'a59ba9e04357114e348ea440317e149b'
Decrypted: Hello
=== AES CFB === 
Cipher:  b'a59ba9e04357114e348ea440317e149b'
Decrypted: Hello
=== AES CTR === 
Cipher:  b'a59ba9e04357114e348ea440317e149b'
Decrypted: Hello


> What is the size of the IV?
> What is the size of the key used?
> For each run, do you get a different cipher for the same input?

## Stream ciphers
With a stream cipher, we take the salt value and some randomization, and create an infinitely long key stream. With this key, we then XOR each bit of the plaintext stream with the key stream. This makes the cipher much faster than a block cipher, and where the main overhead is the key stream generation:


 ![Alt text](graphics/g_sym_05.png "Symmetric Key")

One of the main methods that we use for a stream cipher is ChaCha20, and which provides an alterative to AES. 

In [15]:
# 02_03.py
# https://asecuritysite.com/symmetric/hashnew4
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes 
from cryptography.hazmat.primitives import padding
import sys
from cryptography.hazmat.backends import default_backend

import binascii

def go_encrypt(msg,method,mode):
  cipher = Cipher(method, mode,backend=default_backend())
  encryptor = cipher.encryptor()
  ct = encryptor.update(msg) + encryptor.finalize()
  return (ct)


def go_decrypt(ct,method,mode):
  cipher = Cipher(method, mode,backend=default_backend())
  decryptor = cipher.decryptor()
  return (decryptor.update(ct) + decryptor.finalize())

def go_encrypt_with_auth(msg,method,mode,add):
  cipher = Cipher(method, mode,backend=default_backend())
  encryptor = cipher.encryptor()
  encryptor.authenticate_additional_data(add)
  ct = encryptor.update(msg) + encryptor.finalize()
  return (ct,encryptor.tag)


def go_decrypt_with_auth(ct,method,mode,add):
  cipher = Cipher(method, mode,backend=default_backend())
  decryptor = cipher.decryptor()
  decryptor.authenticate_additional_data(add)
  pl=decryptor.update(ct) + decryptor.finalize()
  return (pl)

def pad(data,size=128):
  padder = padding.PKCS7(size).padder()
  padded_data = padder.update(data)
  padded_data += padder.finalize()
  return(padded_data)

def unpad(data,size=128):
  padder = padding.PKCS7(size).unpadder()
  unpadded_data = padder.update(data)
  unpadded_data += padder.finalize()
  return(unpadded_data)


key = os.urandom(32)
iv = os.urandom(16)
msg=b"Hello"


print ("Message:\t",msg.decode())
print ("Key:\t",binascii.b2a_hex(key))
print ("IV:\t",binascii.b2a_hex(iv))

padded_data=pad(msg)

print ("\n=== ChaCha20 === ")

cipher=go_encrypt(msg,algorithms.ChaCha20(key,iv), None)

plain=go_decrypt(cipher,algorithms.ChaCha20(key,iv), None)

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {plain.decode()}")

Message:	 Hello
Key:	 b'ca472445b83c22523cb4f25ce14c1f21c74294b1672f89a9a570f865794c004a'
IV:	 b'064c1792484091dd65e70ec5a81ea6d9'

=== ChaCha20 === 
Cipher:  b'f36e696aa3'
Decrypted: Hello


> What is the size of the salt value used?

> What is the size of the key used?

> Run the ChaCha20 code, and determine the number of bytes in the cipher for "Hello", "Hello1" and "Hello2". What can you say about the relationship between the plaintext size and the ciphertext size?

## AES GCM 
The most popular symmetric key method is AES GCM, and which is a stream cipher which uses AES encryption. It also adds an authentication tag, so that the cipher can be checked that is have not changed.

 ![Alt text](graphics/g_sym_07.png "Symmetric Key")

The following provides the sample sample code, and where we add in some additional data (AE). This method is known as AEAD (Authenticated Encryption with Additional Data):

In [16]:
# 02_04.py
# https://asecuritysite.com/hazmat/hashnew4
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes 
from cryptography.hazmat.primitives import padding
import sys
from cryptography.hazmat.backends import default_backend

import binascii


def go_encrypt_with_auth(msg,method,mode,add):
  cipher = Cipher(method, mode,backend=default_backend())
  encryptor = cipher.encryptor()
  encryptor.authenticate_additional_data(add)
  ct = encryptor.update(msg) + encryptor.finalize()
  return (ct,encryptor.tag)


def go_decrypt_with_auth(ct,method,mode,add):
  cipher = Cipher(method, mode,backend=default_backend())
  decryptor = cipher.decryptor()
  decryptor.authenticate_additional_data(add)
  pl=decryptor.update(ct) + decryptor.finalize()
  return (pl)



key = os.urandom(32)
iv = os.urandom(16)
msg=b"Hello"
tag= b"Some data for the authentication tag"


print ("Message:\t",msg.decode())
print ("Key:\t",binascii.b2a_hex(key))
print ("IV:\t",binascii.b2a_hex(iv))

padded_data=pad(msg)



print ("=== AES GCM === ")
# In GCM mode we convert to a stream cipher, so there is no need for padding
cipher,auth_tag=go_encrypt_with_auth(msg,algorithms.AES(key), modes.GCM(iv),tag)

plain=go_decrypt_with_auth(cipher,algorithms.AES(key), modes.GCM(iv,auth_tag),tag)

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {plain.decode()}")

Message:	 Hello
Key:	 b'3459f49cb7b8ae9b1dcbb390525ff6b9add7fa0f377ea28f3fff8d28b7b097e5'
IV:	 b'e41b966e55930acd56d5e3dea466d09c'
=== AES GCM === 
Cipher:  b'b4c55b4769'
Decrypted: Hello


> Verify that the cipher changes for each run.

> What do we not need to pad in this code?

> If you change the tag before the decryption, does the cipher decrypt?

 
 ## Additional Ciphers
 There are many other symmetric key methods supported in the cryptography library. This includes  implement AES, Chacha20, Camellia, 3DES, IDEA, CAST5 and Blowfish. The following Python program will use a random 256-bit encryption key, and a random 128-bit IV value. In most of the modes, we need to pad the plaintext to the size of the block (typically this is 128 bits or 16 bytes), but AES GCM mode we do not need to pad the cipher as it is a stream cipher.

In [17]:
# 02_05.py
# https://asecuritysite.com/hazmat/hashnew4
import os
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes 
from cryptography.hazmat.primitives import padding
import sys
from cryptography.hazmat.backends import default_backend

import binascii

def go_encrypt(msg,method,mode):
  cipher = Cipher(method, mode,backend=default_backend())
  encryptor = cipher.encryptor()
  ct = encryptor.update(msg) + encryptor.finalize()
  return (ct)


def go_decrypt(ct,method,mode):
  cipher = Cipher(method, mode,backend=default_backend())
  decryptor = cipher.decryptor()
  return (decryptor.update(ct) + decryptor.finalize())

def go_encrypt_with_auth(msg,method,mode,add):
  cipher = Cipher(method, mode,backend=default_backend())
  encryptor = cipher.encryptor()
  encryptor.authenticate_additional_data(add)
  ct = encryptor.update(msg) + encryptor.finalize()
  return (ct,encryptor.tag)


def go_decrypt_with_auth(ct,method,mode,add):
  cipher = Cipher(method, mode,backend=default_backend())
  decryptor = cipher.decryptor()
  decryptor.authenticate_additional_data(add)
  pl=decryptor.update(ct) + decryptor.finalize()
  return (pl)

def pad(data,size=128):
  padder = padding.PKCS7(size).padder()
  padded_data = padder.update(data)
  padded_data += padder.finalize()
  return(padded_data)

def unpad(data,size=128):
  padder = padding.PKCS7(size).unpadder()
  unpadded_data = padder.update(data)
  unpadded_data += padder.finalize()
  return(unpadded_data)


key = os.urandom(32)
iv = os.urandom(16)
msg=b"Hello"
tag= b"Some data for the authentication tag"


print ("Message:\t",msg.decode())
print ("Key:\t",binascii.b2a_hex(key))
print ("IV:\t",binascii.b2a_hex(iv))

padded_data=pad(msg)


print ("\n\n=== AES ECB === ")
cipher=go_encrypt(padded_data,algorithms.AES(key), modes.ECB())

plain=go_decrypt(cipher,algorithms.AES(key), modes.ECB())
data=unpad(plain)

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")

cipher=go_encrypt(padded_data,algorithms.AES(key), modes.CBC(iv))

print ("=== AES CBC === ")
cipher=go_encrypt(padded_data,algorithms.AES(key), modes.CBC(iv))

plain=go_decrypt(cipher,algorithms.AES(key), modes.CBC(iv))
data=unpad(plain)

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")

cipher=go_encrypt(padded_data,algorithms.AES(key), modes.CBC(iv))

print ("=== AES OFB === ")
cipher=go_encrypt(padded_data,algorithms.AES(key), modes.OFB(iv))

plain=go_decrypt(cipher,algorithms.AES(key), modes.OFB(iv))
data=unpad(plain)

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")

print ("=== AES CFB === ")
cipher=go_encrypt(padded_data,algorithms.AES(key), modes.CFB(iv))

plain=go_decrypt(cipher,algorithms.AES(key), modes.CFB(iv))
data=unpad(plain)

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")

print ("=== AES CTR === ")
cipher=go_encrypt(padded_data,algorithms.AES(key), modes.CTR(iv))

plain=go_decrypt(cipher,algorithms.AES(key), modes.CTR(iv))
data=unpad(plain)

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")

print ("=== AES GCM === ")
# In GCM mode we convert to a stream cipher, so there is no need for padding
cipher,auth_tag=go_encrypt_with_auth(msg,algorithms.AES(key), modes.GCM(iv),tag)

plain=go_decrypt_with_auth(cipher,algorithms.AES(key), modes.GCM(iv,auth_tag),tag)

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {plain.decode()}")


print ("=== AES XTS === ")
# In XTS the iv value is known as a tweak - and relates to the sector number
# The keys are double length, so that a 32 byte key (256 bits) is actually AES-128
cipher=go_encrypt(padded_data,algorithms.AES(key), modes.XTS(iv))

plain=go_decrypt(cipher,algorithms.AES(key), modes.XTS(iv))
data=unpad(plain)
print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")


print ("\n=== Blowfish ECB === ")

cipher=go_encrypt(padded_data,algorithms.Blowfish(key), modes.ECB())

plain=go_decrypt(cipher,algorithms.Blowfish(key), modes.ECB())

data=unpad(plain)

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")

print ("=== Blowfish CBC === ")

cipher=go_encrypt(padded_data,algorithms.Blowfish(key), modes.CBC(iv[:8]))

plain=go_decrypt(cipher,algorithms.Blowfish(key), modes.CBC(iv[:8]))

data=unpad(plain)

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")


print ("\n=== ChaCha20 === ")

cipher=go_encrypt(msg,algorithms.ChaCha20(key,iv), None)

plain=go_decrypt(cipher,algorithms.ChaCha20(key,iv), None)

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")

print ("\n=== 3DES ECB === ")
cipher=go_encrypt(padded_data,algorithms.TripleDES(key[:16]), modes.ECB())

plain=go_decrypt(cipher,algorithms.TripleDES(key[:16]), modes.ECB())

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")

print ("=== 3DES CBC === ")
cipher=go_encrypt(padded_data,algorithms.TripleDES(key[:16]), modes.CBC(iv[:8]))

plain=go_decrypt(cipher,algorithms.TripleDES(key[:16]), modes.CBC(iv[:8]))

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")


print ("\n=== Camellia ECB === ")
cipher=go_encrypt(padded_data,algorithms.Camellia(key), modes.ECB())

plain=go_decrypt(cipher,algorithms.Camellia(key), modes.ECB())

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")


print ("=== Camellia CBC === ")
cipher=go_encrypt(padded_data,algorithms.Camellia(key), modes.CBC(iv))

plain=go_decrypt(cipher,algorithms.Camellia(key), modes.CBC(iv))

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")

print ("=== IDEA ECB === ")
cipher=go_encrypt(padded_data,algorithms.IDEA(key[:16]), modes.ECB())

plain=go_decrypt(cipher,algorithms.IDEA(key[:16]), modes.ECB())

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")

print ("=== IDEA CBC === ")
cipher=go_encrypt(padded_data,algorithms.IDEA(key[:16]), modes.CBC(iv[:8]))

plain=go_decrypt(cipher,algorithms.IDEA(key[:16]), modes.CBC(iv[:8]))

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")

print ("\n=== CAST5 ECB === ")
cipher=go_encrypt(padded_data,algorithms.CAST5(key[:16]), modes.ECB())

plain=go_decrypt(cipher,algorithms.CAST5(key[:16]), modes.ECB())

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")

print ("=== CAST5 CBC === ")
cipher=go_encrypt(padded_data,algorithms.CAST5(key[:16]), modes.CBC(iv[:8]))

plain=go_decrypt(cipher,algorithms.CAST5(key[:16]), modes.CBC(iv[:8]))

print ("Cipher: ",binascii.b2a_hex(cipher))
print (f"Decrypted: {data.decode()}")

Message:	 Hello
Key:	 b'394f66d40b7ff08e054f155d581299feeee5dd4c346f455432cf280003b65cc9'
IV:	 b'783992db351505f6e4455ece244288b1'


=== AES ECB === 
Cipher:  b'8d18d2608555aafa6c49a867a3df7da3'
Decrypted: Hello
=== AES CBC === 
Cipher:  b'e883fc6c0b568e984f37b5be0660990d'
Decrypted: Hello
=== AES OFB === 
Cipher:  b'39dccf1ec235d002ca472abe03e9cc98'
Decrypted: Hello
=== AES CFB === 
Cipher:  b'39dccf1ec235d002ca472abe03e9cc98'
Decrypted: Hello
=== AES CTR === 
Cipher:  b'39dccf1ec235d002ca472abe03e9cc98'
Decrypted: Hello
=== AES GCM === 
Cipher:  b'38426fbe32'
Decrypted: Hello
=== AES XTS === 
Cipher:  b'dbc50e1f568ac761307228c06a11c6a2'
Decrypted: Hello

=== Blowfish ECB === 
Cipher:  b'0d3166526e87a287e20dbbca5068a253'
Decrypted: Hello
=== Blowfish CBC === 
Cipher:  b'bdb3d83379e84e42de2c68cfc6100cd8'
Decrypted: Hello

=== ChaCha20 === 
Cipher:  b'3aa44314ab'
Decrypted: Hello

=== 3DES ECB === 
Cipher:  b'ee9fbc2a9e2ab17d9294e35946806f88'
Decrypted: Hello
=== 3DES CBC === 
Cipher:  

# Computing time to crack symmetric key with brute force
If we have x bits for a symmetric key, we will then have 2^x different keys. If we assume that it takes t seconds to try a single key, then the total time to crack will be:

T = 2^x * t

Overall, the average time will be half of this value. For example, if we have a 32-bit key and can crack for 1 nanosecond per key, we can compute the average time with:

In [8]:
x=32
keys=2**x
t=1e-9

time=keys*t
print(f"Time to crack = {time/2} seconds")
print(f"Time to crack = {time/2/60/60} hours")

Time to crack = 18446744073.709553 seconds
Time to crack = 213503.9823346013 days


> Modify the program to show the number of days to crack.
> For a 64 bit key, and 1 pico second crack per key, what is the average cracking time in days?